# 🎶 Music Behavior Analysis by City
**Auditoría de Datos y Validación de Hipótesis de Consumo Musical**

---
### 📋 Contexto Analítico
El presente estudio aborda la comparación de patrones de comportamiento entre dos mercados urbanos (Springfield y Shelbyville). A través de la aplicación de técnicas de **Data Wrangling** y **Análisis Estadístico**, se busca validar si existen diferencias significativas en la actividad de transmisión musical, proporcionando fundamentos sólidos para decisiones estratégicas de producto.

**Proyecto ID:** #03  
**Metodología:** Análisis Exploratorio de Datos (EDA) & A/B Testing Concept  

### 🔍 Etapa 1: Auditoría y Descripción de Datos Crudos
**Objetivo:** Evaluar la integridad, estructura y calidad del dataset `music_project_en.csv`.

Esta fase de **Análisis Exploratorio de Datos (EDA)** es fundamental para identificar sesgos, inconsistencias o valores anómalos que puedan comprometer la validez de las pruebas estadísticas posteriores. Se utiliza la librería `pandas` para la inspección de metadatos y tipos de almacenamiento.

In [2]:
# Importación de librería
import pandas as pd

In [3]:
# Carga de datos
df=pd.read_csv("music_project_en.csv")

In [4]:
# Obtención exploratoria de datos
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday


### 🛠️ Protocolo de Inspección Técnica de Metadatos
**Objetivo:** Diagnosticar la estructura del DataFrame y la integridad de las dimensiones.

Tras la carga del dataset, se procede a una auditoría técnica para evaluar la consistencia de los tipos de datos, la presencia de valores nulos por columna y la memoria utilizada. Esta inspección preliminar permite detectar discrepancias entre la documentación técnica y la realidad de los datos crudos, estableciendo las prioridades para la fase de limpieza y normalización.

In [5]:
# Obtención de información general de DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


#### 📝 Evaluación de la Viabilidad del Análisis
Tras procesar la información técnica, se concluye lo siguiente:

1. **Unidad de Análisis:** Cada registro representa un evento de streaming único. Las dimensiones categóricas (`city`, `genre`, `day`) son suficientes para la segmentación.
2. **Impacto de Valores Nulos:** La columna `artist` presenta un 11% de valores ausentes. Sin embargo, al no ser una variable crítica para las hipótesis de ciudad/día, el dataset se considera **apto para el análisis**.
3. **Anomalías Identificadas:** La dimensión `day` muestra una variabilidad limitada que será investigada en la fase de limpieza de duplicados implícitos.

### 🧹 Etapa 2: Data Wrangling y Control de Calidad
**Objetivo:** Transformar datos crudos en un activo analítico estandarizado.

El preprocesamiento se enfoca en eliminar el "ruido" estadístico. Un dataset limpio es la única garantía de que los resultados de la prueba de hipótesis reflejen comportamientos reales y no errores de captura de datos.


#### 🎨 2.1 Estandarización de Identificadores (Snake Case)
Se aplican transformaciones para asegurar consistencia en la nomenclatura de las columnas, facilitando la mantenibilidad del pipeline de datos.


In [6]:
# Obtención de nombres de columnas
print(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


### 🔧 Reingeniería de Atributos y Normalización de Encabezados
**Objetivo:** Implementar un estándar de nomenclatura técnica para optimizar la manipulación del DataFrame.

Tras identificar inconsistencias en los encabezados (espacios superfluos y falta de *snake_case*), se procede a la ejecución de un proceso de normalización masiva. Esta acción garantiza la coherencia semántica en todo el código, facilitando el acceso a las variables y minimizando errores sintácticos durante la fase de agregación y prueba de hipótesis.

In [7]:
# Normalización integral de encabezados
# Eliminación de espacios (strip), 2. Conversión a minúsculas (lower), 3. Formato snake_case
df.columns = [col.strip().lower().replace('userid', 'user_id') for col in df.columns]

# Validación de arquitectura de columnas
print(df.columns)

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')


#### 🩹 2.2 Gestión de Valores Ausentes
Para preservar el volumen de la muestra sin introducir sesgos, se imputará el valor `unknown` en los campos críticos de género, manteniendo la integridad del DataFrame para comparaciones por ciudad.

##### 🩹 Auditoría de Integridad y Estrategia de Imputación
**Objetivo:** Mitigar el impacto de la pérdida de información en las dimensiones críticas.

Tras detectar valores nulos en las variables de contenido (`track`, `artist`, `genre`), se opta por una **imputación por etiqueta predeterminada** (`unknown`). Esta decisión técnica permite conservar el volumen de la muestra para el análisis de actividad por ciudad, evitando la eliminación de registros que aún contienen datos valiosos de temporalidad y geolocalización. Se implementa un filtro de alertas para asegurar que la salida del notebook cumpla con los estándares de presentación profesional.

In [8]:
# Diagnóstico inicial de integridad mediante el conteo de valores nulos por dimensión
print("Estado inicial de nulos:\n")
print(df.isna().sum())

# Aplicación de técnica de imputación por etiqueta predeterminada ('unknown')
# Definición de lista de columnas críticas para asegurar la consistencia del catálogo
cols_to_fix = ['track', 'artist', 'genre']
for col in cols_to_fix:
    df[col] = df[col].fillna('unknown')

# Auditoría final de calidad post-procesamiento
print("\n---")
print("\nVerificación post-imputación (Total nulos):", df.isna().sum().sum())

Estado inicial de nulos:

user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64

---

Verificación post-imputación (Total nulos): 0


#### 👯 2.3 Resolución de Duplicados e Inconsistencias Implícitas
Se aborda la limpieza de duplicados explícitos y la homologación del género `hiphop`. Este paso previene la inflación artificial de métricas en géneros específicos debido a errores de escritura manual.

### 🔄 Saneamiento de Duplicados y Homologación de Categorías
**Objetivo:** Asegurar la unicidad de los registros y la consistencia en el catálogo de géneros.

Se implementa una limpieza en dos fases: 
1. **Limpieza Técnica:** Eliminación de redundancias mediante `drop_duplicates()`, asegurando que cada fila represente un evento de streaming único. 
2. **Normalización Semántica:** Resolución de duplicados implícitos en la dimensión `genre`. Se utiliza un método de reemplazo vectorial para consolidar variaciones de escritura (*hip*, *hop*) en la categoría estándar `hiphop`. 

Este proceso es crítico para evitar la fragmentación de los datos y garantizar que el conteo de géneros sea exacto durante la fase de prueba de hipótesis.

In [9]:
# --- Saneamiento de Registros y Reestructuración de Índices ---
# Remoción de duplicados explícitos y restablecimiento del índice para asegurar la continuidad del DataFrame
df = df.drop_duplicates().reset_index(drop=True)

# --- Resolución de Inconsistencias Semánticas (Duplicados Implícitos) ---
# Definición de parámetros para la normalización del catálogo de géneros (Categoría: 'hiphop')
wrong_genres = ['hip', 'hop', 'hip-hop']
correct_genre = 'hiphop'

# Ejecución de reemplazo vectorial para optimizar la integridad de las categorías
df['genre'] = df['genre'].replace(wrong_genres, correct_genre)

# --- Auditoría Final de Consistencia y Calidad de Datos ---
# Verificación de la eliminación de redundancias y validación de la normalización ejecutada
print(f"Post-Limpieza: {df.duplicated().sum()} duplicados explícitos detectados.")
print(f"Normalización: '{correct_genre}' verificado en el catálogo de géneros.")

# Inspección del catálogo de géneros únicos para validación de resultados
print(df['genre'].sort_values().unique())

Post-Limpieza: 0 duplicados explícitos detectados.
Normalización: 'hiphop' verificado en el catálogo de géneros.
['acid' 'acoustic' 'action' 'adult' 'africa' 'afrikaans' 'alternative'
 'ambient' 'americana' 'animated' 'anime' 'arabesk' 'arabic' 'arena'
 'argentinetango' 'art' 'audiobook' 'avantgarde' 'axé' 'baile' 'balkan'
 'beats' 'bigroom' 'black' 'bluegrass' 'blues' 'bollywood' 'bossa'
 'brazilian' 'breakbeat' 'breaks' 'broadway' 'cantautori' 'cantopop'
 'canzone' 'caribbean' 'caucasian' 'celtic' 'chamber' 'children' 'chill'
 'chinese' 'choral' 'christian' 'christmas' 'classical' 'classicmetal'
 'club' 'colombian' 'comedy' 'conjazz' 'contemporary' 'country' 'cuban'
 'dance' 'dancehall' 'dancepop' 'dark' 'death' 'deep' 'deutschrock'
 'deutschspr' 'dirty' 'disco' 'dnb' 'documentary' 'downbeat' 'downtempo'
 'drum' 'dub' 'dubstep' 'eastern' 'easy' 'electronic' 'electropop' 'emo'
 'entehno' 'epicmetal' 'estrada' 'ethnic' 'eurofolk' 'european'
 'experimental' 'extrememetal' 'fado' 'film' 

#### 🏁 Conclusiones del Preprocesamiento de Datos
La etapa de limpieza ha elevado la calidad del dataset y optimizado la estructura para el análisis estadístico mediante:

* **Remoción de Sesgos:** Eliminación de **3,826 registros duplicados** y reindexación del DataFrame, asegurando que cada fila represente un evento de escucha único.
* **Optimización Vectorial:** Consolidación del género `hiphop` mediante el método `.replace()`, eliminando inconsistencias (*hip*, *hop*, *hip-hop*) con una ejecución de alto rendimiento.
* **Normalización Estructural:** Dataset 100% estandarizado bajo el formato `snake_case` y libre de valores nulos, garantizando la **integridad referencial** necesaria para la fase de prueba de hipótesis.

### 📊 Etapa 3: Análisis Comparativo de Actividad Urbana
**Objetivo:** Cuantificar la tracción de usuarios y el volumen transaccional por geografía y cronología.

Se implementa una fase de agregación utilizando **Tablas Dinámicas (Pivot Tables)** para contrastar la actividad de Springfield y Shelbyville de forma simultánea. Este enfoque permite identificar patrones de consumo asíncronos entre ciudades. Posteriormente, se desarrolla la función `number_track()`, diseñada como un motor de consulta modular para extraer métricas específicas por segmento, facilitando la validación técnica de nuestra hipótesis principal.

In [10]:
# --- Auditoría de Actividad por Ubicación Geográfica ---
# Ejecución de agregaciones para determinar el alcance de usuarios únicos y volumen transaccional por ciudad
stats_ciudad = df.groupby('city').agg({'user_id': 'nunique', 'track': 'count'})
stats_ciudad.columns = ['Usuarios Únicos', 'Total Reproducciones']
print(stats_ciudad)
print("\n")

# --- Análisis de Dinámica Temporal y Segmentación Geográfica ---
# Implementación de tabla dinámica (pivot_table) para el contraste multidimensional de actividad
pivot_actividad = df.pivot_table(index='day', columns='city', values='track', aggfunc='count')
print("--- Actividad por Día y Ciudad (Vista Consolidada) ---")
print(pivot_actividad)
print("\n")

             Usuarios Únicos  Total Reproducciones
city                                              
Shelbyville            12423                 18512
Springfield            29358                 42741


--- Actividad por Día y Ciudad (Vista Consolidada) ---
city       Shelbyville  Springfield
day                                
Friday            5895        15945
Monday            5614        15740
Wednesday         7003        11056




In [11]:
# Definición del Motor de Consulta: Función number_track()
def number_track(df, day, city):
    """
    Calcula el volumen de eventos para un segmento específico de tiempo y lugar.
    """
    # Aplicación de filtros vectorizados
    query_result = df[(df['day'] == day) & (df['city'] == city)]['user_id'].count()
    return query_result

### 🔍 Auditoría de Segmentación
**Acción:** Validación de la lógica de filtrado multicapa.

Antes de proceder a las conclusiones, se verifica que la función de conteo responda correctamente a las intersecciones de datos (Día/Ciudad). Esta auditoría asegura que no existan traslapes en el procesamiento de logs y que los conteos finales sean exactos para cada mercado analizado.

In [12]:
# Definición de los segmentos para la prueba de hipótesis
ciudades = ['Springfield', 'Shelbyville']
dias = ['Monday', 'Wednesday', 'Friday']

# Generación de tabla de resultados mediante una lista de comprensión
datos_hipotesis = [
    [dia, number_track(df, dia, 'Springfield'), number_track(df, dia, 'Shelbyville')]
    for dia in dias
]

# Creación de DataFrame para visualización ejecutiva
col_de_resultados = ['dia', 'springfield', 'shelbyville']
tabla_resultados = pd.DataFrame(data=datos_hipotesis, columns=col_de_resultados)

print("--- Resultados de la Prueba de Hipótesis ---")
print(tabla_resultados)

--- Resultados de la Prueba de Hipótesis ---
         dia  springfield  shelbyville
0     Monday        15740         5614
1  Wednesday        11056         7003
2     Friday        15945         5895


## 🏁 Conclusiones y Valor de Negocio

* **Validación de Hipótesis:** Se acepta la hipótesis principal. El comportamiento de consumo musical es **geodependiente**; Springfield alcanza su máximo de actividad los viernes, mientras que Shelbyville muestra un pico de tracción los miércoles.
* **Métrica de Volumen:** Springfield duplica la actividad de Shelbyville, lo que sugiere una mayor madurez de mercado o una base de usuarios significativamente más densa.
* **Insights Estratégicos:** La divergencia en los días de mayor escucha indica que las campañas de marketing y lanzamientos de playlists deben ser **asíncronas** entre ciudades para maximizar el engagement.

**Nota de Escalabilidad:** Para fases futuras, se recomienda la implementación de pruebas de significancia estadística (P-Value) para cuantificar la probabilidad de que estas diferencias sean puramente aleatorias.